Material adapted from https://kodu.ut.ee/~kmoch/geopython2019/L4/raster.html
and http://patrickgray.me/open-geo-tutorial/chapter_3_visualization.html


Imports

In [ ]:
import geopandas as gpd
import rasterio, descartes, ipyleaflet
import os
import fiona
import numpy as np
import matplotlib.pyplot as plt
from rasterio.plot import show
from rasterio.plot import show_hist
from rasterio.mask import mask

Set working directory

In [ ]:
wdir = r'G:/My Drive/Projects/geode/tutorials/pythontutorial'

Set file names

In [ ]:
# Year 2000 intact forest landscapes
iflPath2000 = 'ifl_239_276_2000.shp'
# Year 2020 intact forest landscapes
iflPath2020 = 'ifl_239_276_2020.shp'

# Year 2000 dispersal probability surface
dpsPath2000 = 'corr_2000_239_276.tif'
# Year 2020 dispersal probability surface
dpsPath2020 = 'corr_2020_239_276.tif'

# Protected areas
pasPath = 'bz_pa.shp'

Read a tif of the probability of dispersal between two intact forest landscapes in the year 2000

In [ ]:
ds1 = rasterio.open(wdir + '/' + dpsPath2000)

Describe it

In [ ]:
print(ds1.name)
print(ds1.width)
print(ds1.height)
print(ds1.crs)
print(ds1.bounds)

In [ ]:
# A more comprehensive description
print(ds1.profile)

Each raster dataset can have several bands. Each band in Python and Rasterio is essentially handled as a Numpy array.

In [ ]:
# Read the first band to a variable and print
dps00 = ds1.read(1)
dps00

In [ ]:
# Plot the first band
show(dps00)

In [ ]:
# Calculate summary stats of dispersal probabilities
print('min is ' + str(np.min(dps00)))
print('max is ' + str(np.max(dps00)))
print('mean is ' + str(np.mean(dps00)))
print('mean in year 2000 corridor is ' + str(np.mean(dps00[dps00 > 0])))

In [ ]:
# Plot histogram of raster values
show_hist(ds1, bins=19, lw=0.0, stacked=False, alpha=0.3, histtype='stepfilled', title="Histogram")

In [ ]:
# Read in year 2020 dispersal probability surface
ds2 = rasterio.open(wdir + '/' + dpsPath2020)
dps20 = ds2.read(1)
show(dps20)
print('mean in year 2000 corridor is ' + str(np.mean(dps00[dps00 > 0])))
print('mean in year 2020 corridor is ' + str(np.mean(dps20[dps20 > 0])))

Plot side by side

In [ ]:
# 2by plot using matplotlib object oriented approach
fig, (ax1, ax2) = plt.subplots(1,2, figsize=(10,8)) # 2 axes on a 1x2 grid
# find max probability to put them on the same colorbar scale
max_prob = np.amax([np.amax(dps00), np.amax(dps20)])
# 2000 probs
dps00ax = ax1.imshow(dps00)
ax1.set_title("Dispersal Probability 2000")
dps00ax.set_clim(vmin=0, vmax=max_prob)
fig.colorbar(dps00ax, ax=ax1)

# 2020 probs
dps20ax = ax2.imshow(dps20)
ax2.set_title("Dispersal Probability 2020")
dps20ax.set_clim(vmin=0, vmax=max_prob)
fig.colorbar(dps20ax, ax=ax2)

In [ ]:
# Plot IFLs over corridor
iflPoly00 = gpd.read_file(wdir + '/' + iflPath2000)
fig, ax = plt.subplots(1, figsize=(10, 8))
show((ds1, 1), cmap='terrain', interpolation='none', ax=ax)
# Need descartes for this
iflPoly00.plot(ax=ax, facecolor="none", edgecolor='black', lw=0.7)

The intact forest landscapes connected by this corridor are mostly protected. What is the probability of dispersal like in those protected areas?

In [ ]:
#  Read in PAs and print.
pas = gpd.read_file(wdir + '/' + pasPath)
pas


The fiona library works actually under the hood of geopandas. With fiona we can open vector/feature datasets directly without loading them into a dataframe. This is required for masking rasters.

In [ ]:
# Read ifls to geoms and print
with fiona.open(wdir + '/' + iflPath2000, "r") as shapefile:
    geoms = [feature["geometry"] for feature in shapefile]
geoms

In [ ]:
# Plot PAs over corridor
fig, ax = plt.subplots(1, figsize=(10, 8))
show((ds1, 1), cmap='terrain', interpolation='none', ax=ax)
# Need descartes for this
pas.plot(ax=ax, facecolor="none", edgecolor='black', lw=0.7)

In [ ]:
# Crop year 2000 dispersal probability surface to PAs
dps00inPA, dps00inPATransform = mask(dataset=ds1, shapes=geoms, crop=True)
print('year 2000 mean dp in PA is ' + str(np.mean(dps00inPA[dps00inPA > 0])))
# Crop year 2020 dispersal probability surface to PAs
dps20inPA, dps20inPATransform = mask(dataset=ds2, shapes=geoms, crop=True)
print('year 2020 mean dp in PA is ' + str(np.mean(dps20inPA[dps20inPA > 0])))

In [ ]:
# Invert mask on year 2000 dispersal probability surface to grab values outside PAs
dps00notinPA, dps00notinPATransform = mask(dataset=ds1, shapes=geoms, invert=True, crop=False)
print('year 2000 mean dp outside PA is ' + str(np.mean(dps00notinPA[dps00notinPA > 0])))
# Invert mask on year 2020 dispersal probability surface to grab values outside PAs
dps20notinPA, dps20notinPATransform = mask(dataset=ds2, shapes=geoms, invert=True, crop=False)
print('year 2020 mean dp outside PA is ' + str(np.mean(dps20notinPA[dps20notinPA > 0])))
# Need to manually close file objects
ds1.close()
ds2.close()

In [ ]:
# Percent decrease inside PA
(0.7139251-0.688464)/0.7139251*100

In [ ]:
# Percent decrease outside PA
(0.7521234-0.7171582)/0.7521234*100
print(dpsPath2000.replace('.tif','_masked.tif'))

What if we want to save a masked array?

In [ ]:
# Use existing fiona geometry objects
# Use with context manager (automatically closes file objects)
# Use existing tiff as template
with rasterio.open(wdir + '/' + dpsPath2000) as src:
    out_meta = src.meta
    out_image, out_transform = rasterio.mask.mask(src, shapes=geoms, crop=True)

    profile = src.profile
    profile["height"] = out_image.shape[1]
    profile["width"] = out_image.shape[2]
    profile["transform"] = out_transform

    out_meta.update({"driver": "GTiff",
                 "height": out_image.shape[1],
                 "width": out_image.shape[2],
                 "transform": out_transform})

with rasterio.open(wdir + '/' + dpsPath2000.replace('.tif','_masked.tif'), "w", **out_meta) as dest:
    dest.write(out_image)

# Pathlib is a useful module. Here use to check if file was created.
from pathlib import Path
Path(wdir + '/' + dpsPath2000.replace('.tif','_masked.tif')).exists()

In [ ]:
# What if we have categorical data?
# Convert probability values to ranks
bins = [0, 0.25, 0.5, 0.75, 1]
probRanks00 = np.digitize(dps00, bins)
# Calculate rank counts
unique, counts = np.unique(probRanks00, return_counts=True)
print(np.asarray((unique, counts)).T)


In [ ]:
# Lowest rank is inflated due to fill values
probRanks00 = np.digitize(dps00[dps00 > 0], bins)
# Calculate rank counts
unique, counts = np.unique(probRanks00, return_counts=True)
print(np.asarray((unique, counts)).T)

Visualize with Leaflet

In [ ]:
from ipyleaflet import Map, basemaps
Map(center = (60, -2.2), zoom = 2, min_zoom = 1, max_zoom = 20, 
    basemap=basemaps.Stamen.Terrain)


In [ ]:
# Need lat lon for basemap
pas_repro  = pas.to_crs({'init': 'epsg:4326'})
# Returns a tuple containing minx, miny, maxx, maxy
bbox = pas_repro.total_bounds
print(bbox)

# Use bounding box coords to center map
#map = Map(center = (53.48, -2.24), basemap=basemaps.Esri.WorldImagery)
map = Map(center = (np.mean([bbox[1],bbox[3]]),np.mean([bbox[0],bbox[2]])), basemap=basemaps.Esri.WorldImagery)
map